### 1. 키워드파일 업로드

In [7]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# CSV 파일에서 클러스터링 결과 로드
df_inheritance = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')

### 2. 사건명가져오기 위해서 판례목록 데이터 불러오기

In [8]:
# 사건 리스트 로드
inheritance_list = pd.read_csv('inheritance_list.csv')

# 사건명 추가를 위해 df_inheritance 데이터프레임과 병합
df_inheritance = df_inheritance.merge(inheritance_list[['사건번호', '사건명']], on='사건번호', how='left')

In [9]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166 entries, 0 to 4165
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       4166 non-null   int64 
 1   사건번호           4166 non-null   object
 2   선고             4166 non-null   object
 3   판결유형           4166 non-null   object
 4   판시사항           3250 non-null   object
 5   판결요지           2932 non-null   object
 6   참조조문           3225 non-null   object
 7   참조판례           1935 non-null   object
 8   판례내용           4166 non-null   object
 9   판례내용_상단        4166 non-null   object
 10  판례내용_이유        4166 non-null   object
 11  판례내용_이유_전처리    4146 non-null   object
 12  판례내용_이유_불용어제거  4131 non-null   object
 13  클러스터           4166 non-null   int64 
 14  키워드            4166 non-null   object
 15  사건명            3924 non-null   object
dtypes: int64(2), object(14)
memory usage: 520.9+ KB


### 3. 임베딩해서 csv파일로 저장하기

In [10]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용과 클러스터 키워드 임베딩 생성
df_inheritance['판례내용이유임베딩'] = df_inheritance['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['키워드임베딩'] = df_inheritance['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['판결요지임베딩'] = df_inheritance['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['판시사항임베딩'] = df_inheritance['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_inheritance.to_csv('df_inheritance_kobertmodel.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [29]:
df_inheritance.head(2)

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,...,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드,사건명,판례내용이유임베딩,키워드임베딩,판결요지임베딩,판시사항임베딩,키워드임베딩수정
0,238913,2023다221144,선고,판결,"[1] 가분채권이 공동상속되는 경우의 법률관계 / 주식이 공동상속되는 경우, 공동...",[1] 금전채권과 같이 급부의 내용이 가분인 채권은 공동상속되는 경우 상속개시와 ...,"[1] 민법 제269조, 제278조, 제547조 제1항, 제1005조, 제1006...","[1] 대법원 2003. 5. 30. 선고 2003다7074 판결, 대법원 200...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...",...,상고이유를 판단한다. 1. 사안의 개요 가. 사실관계 원심판결 이유와 기록에 따르면...,개요 사실관계 망인 은행 예금 채권 은행 판매 투자신탁 형태 MMF Money Ma...,2,"신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주",예금[투자신탁 형태 단기금융집합투자기구(MMF) 수익권의 공동상속 효과가 문제된 사건],"[[0.09682950377464294, -0.2833014726638794, 0....","[[0.08193794637918472, 0.05260368064045906, 0....","[[0.12045687437057495, -0.09459716826677322, 0...","[[0.20212049782276154, -0.06857649981975555, 0...","[[[0.0556497760117054, -0.14372481405735016, -..."
1,240673,2021두52143,선고,판결,대한민국에서 주로 체류하며 대한민국에 있는 건축사무소 등에서 건축설계사로 근무하...,NaN,구 섭외사법(2001. 4. 7. 법률 제6465호 국제사법으로 전부 개정되기 전의...,NaN,"【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...","【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...",...,"상고이유를 판단한다. 1. 원심판결 이유와 기록에 따르면, 다음의 사실을 알 수 있...",인과 대한민국 혼인신고 마쳤고 자녀 미합중국 캘리포니아 출생 미합중국 시민권 취...,2,"신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주",상속세부과처분취소,"[[0.1609489917755127, -0.2994731664657593, 0.6...","[[0.08193794637918472, 0.05260368064045906, 0....",None,"[[0.053899895399808884, 0.21528321504592896, 0...","[[[0.0556497760117054, -0.14372481405735016, -..."


#### 나중에 다시할거

In [16]:
# 클러스터 주제 및 키워드 데이터 정의
data = {
    '주제': [
        '상속 및 재산 평가', '부동산 소유권 및 취득', '부동산 상속 및 사망 관련',
        '부동산 등기 및 소유권 이전', '손해배상 및 사망 보상', '농지 및 경작',
        '상속세 및 납부', '부동산 등기 및 거래', '상속 분할 및 유류분', '보험 및 보상'
    ],
    '키워드': [
        ['가액', '시가', '평가', '상속', '세법', '주식', '재산', '과세', '증여', '상속세'],
        ['점유', '토지', '취득시효', '등기', '소유권', '완성', '자주', '부동산', '취득', '의사'],
        ['신청', '상속', '부동산', '주택', '사망', '건물', '계약', '회사', '재산', '호주'],
        ['등기', '임야', '소유권', '부동산', '이전', '명의', '원인', '상속', '명의신탁', '토지'],
        ['망인', '손해', '차량', '위자료', '지급', '손해배상', '사망', '유족', '운전', '운행'],
        ['농지', '분배', '농지개혁법', '토지', '상환', '등기', '본건', '소유권', '경작', '완료'],
        ['상속세', '세액', '부과', '납부', '과세', '상속', '재산', '가액', '신고', '납세'],
        ['토지', '등기', '소유권', '이전', '명의', '분할', '지번', '주소', '환지', '매매'],
        ['상속', '재산', '분할', '유류분', '상속인', '부동산', '민법', '한정승인', '피상', '포기'],
        ['보험', '보험금', '계약', '지급', '망인', '보험료', '자동차', '연금', '수익', '상해']
    ]
}

df_clusterdata = pd.DataFrame(data)

# 키워드를 보여주는 함수
def show_keywords(topic_index):
    for button in buttons:
        button.destroy()
    buttons.clear()

    keywords = df_clusterdata.iloc[topic_index]['키워드']
    for keyword in keywords:
        button = tk.Button(root, text=keyword, command=lambda k=keyword: keyword_click(k))
        button.pack(pady=5)
        buttons.append(button)

# 키워드 클릭 시 호출되는 함수
def keyword_click(keyword):
    user_keyword = keyword
    user_keyword_embedding = encode_sentence(user_keyword)

    similarity_scores = []
    for idx, row in df_inheritance.iterrows():
        if row['키워드임베딩'] is not None and row['이유임베딩'] is not None:
            keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
            content_similarity = cosine_similarity(user_keyword_embedding, row['이유임베딩']).flatten()[0]
            average_similarity = (keyword_similarity + content_similarity) / 2
            similarity_scores.append((row['사건번호'], average_similarity))

    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_n = sorted_scores[:10]

    result = "사용자 키워드와 관련된 사건명:\n"
    for 사건번호, score in top_n:
        사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
        사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
        result += f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}\n"

    messagebox.showinfo("추천 사건명", result)

# 주제 선택 시 호출되는 함수
def topic_selected(event):
    topic_index = df_clusterdata.index[df_clusterdata['주제'] == topic_var.get()].tolist()[0]
    show_keywords(topic_index)

# 메인 윈도우 생성
root = tk.Tk()
root.title("원하는 판례찾기")

# 주제 선택 드롭다운 메뉴 생성
topic_var = tk.StringVar(value='관련 주제를 선택하세요')
topic_menu = tk.OptionMenu(root, topic_var, *df_clusterdata['주제'], command=topic_selected)
topic_menu.pack(pady=10)

# 버튼 리스트 초기화
buttons = []

# 메인 루프 실행
root.mainloop()

In [5]:
# 사용자 입력
user_keyword = "재산"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 유사도 계산 및 사건명 추출
similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['키워드임베딩'] is not None and row['이유임베딩'] is not None:
        # 키워드 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        keyword_similarity = cosine_similarity(user_keyword_embedding, row['키워드임베딩']).flatten()[0]
        
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['이유임베딩']).flatten()[0]
        
        # 두 유사도의 평균을 계산
        average_similarity = (keyword_similarity + content_similarity) / 2
        
        # 결과 저장
        similarity_scores.append((row['사건번호'], average_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
top_n = sorted_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, score in top_n:
    # 사건번호를 통해 사건명 조회
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명:
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.4304
사건번호: 2003다64381, 사건명: 종헌결의무효확인, 유사도: 0.4286
사건번호: 2007카기134, 사건명: 위헌법률심판제청, 유사도: 0.4264
사건번호: 2021나31998, 사건명: 공유물분할, 유사도: 0.4257
사건번호: 2018두32927, 사건명: 취득세등경정청구거부처분취소, 유사도: 0.4248
사건번호: 94머211, 사건명: 소유권이전등기말소청구사건, 유사도: 0.4239
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.4238
사건번호: 2005도5338, 사건명: 특정경제범죄가중처벌등에관한법률위반(횡령)(인정된죄명:업무상횡령)·업무상횡령, 유사도: 0.4237
사건번호: 88다카3847, 사건명: 토지소유권확인등, 유사도: 0.4224
사건번호: 93타경23863, 사건명: 부동산임의경매, 유사도: 0.4222


### 4. 유사도 계산해서 추천사건명 출력하기

#### - 사용자가 입력한 키워드와 유사도로 추천하기

#### 판결요지

In [23]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판결요지임베딩'] is not None:

        content_similarity = cosine_similarity(user_keyword_embedding, row['판결요지임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명:
사건번호: 4294민상168, 사건명: 덕대계약이행완료확인, 유사도: 0.5380
사건번호: 81다카550, 사건명: 소유권이전등기말소, 유사도: 0.5114
사건번호: 86다카1312, 사건명: 소유권이전등기말소등, 유사도: 0.4960
사건번호: 4294민상1255, 사건명: 가옥명도, 유사도: 0.4863
사건번호: 62다343, 사건명: 부동산소유권이전등기말소, 유사도: 0.4846
사건번호: 87다카663, 사건명: 소유권이전등기말소등, 유사도: 0.4839
사건번호: 90다카17214, 사건명: 소유권이전등기말소, 유사도: 0.4827
사건번호: 80구796, 사건명: 양도소득세부과처분취소, 유사도: 0.4824
사건번호: 80구844, 사건명: 취득세부과처분취소, 유사도: 0.4824
사건번호: 90다카3789, 사건명: 소유권이전등기말소등, 유사도: 0.4777


#### 판례내용_이유

In [24]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판례내용이유임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판례내용이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.3974
사건번호: 87누898, 사건명: 증여세등부과처분취소, 유사도: 0.3690
사건번호: 2000으2, 사건명: 상속재산관리인선임, 유사도: 0.3683
사건번호: 83누221, 사건명: 상속세부과처분취소, 유사도: 0.3666
사건번호: 2012누8801, 사건명: 상속세부과처분취소, 유사도: 0.3662
사건번호: 2009브23, 사건명: 상속한정승인결정에대한즉시항고, 유사도: 0.3646
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.3645
사건번호: 79다1009, 사건명: 소유권보존등기말소, 유사도: 0.3628
사건번호: 84누649, 사건명: 상속세부과처분취소, 유사도: 0.3625
사건번호: 2006다2179, 사건명: 상속분양수, 유사도: 0.3621


#### 판시사항

In [25]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판시사항임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판시사항임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 95누9327, 사건명: 상속세부과처분취소, 유사도: 0.8622
사건번호: 2007다9719, 사건명: 소유권이전등기, 유사도: 0.8622
사건번호: 85누804, 사건명: 증여세부가처분취소, 유사도: 0.8622
사건번호: 83다카893, 사건명: 부당이득금, 유사도: 0.8622
사건번호: 83다카745, 사건명: 대여금, 유사도: 0.8622
사건번호: 82누138, 사건명: 양도소득세부과처분취소, 유사도: 0.8622
사건번호: 81나459, 사건명: 공유물분할청구사건, 유사도: 0.8622
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.8622
사건번호: 80나4240, 사건명: 손해배상청구사건, 유사도: 0.8622
사건번호: 79다434, 사건명: 소유권보존등기말소, 유사도: 0.8622
